In [1]:
from gevent import monkey
monkey.patch_all()
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./2.menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        for b in range(666):
            try:
                import requests

                import sys
                sys.path.append('../../00.Tools')
                from crawler_configuration import get_proxy

                if input_.loc[a, 'Market'] == 'US':
                    url = f'''https://www.ebay.com/itm/{input_.loc[a, 'Item Number']}?_ul=US&_stpos=91710&orig_cvip=true'''
                elif input_.loc[a, 'Market'] == 'DE':
                    url = f'''https://www.ebay.de/itm/{input_.loc[a, 'Item Number']}?_ul=DE&_stpos=10115&orig_cvip=true'''
                elif input_.loc[a, 'Market'] == 'UK':
                    url = f'''https://www.ebay.co.uk/itm/{input_.loc[a, 'Item Number']}?_ul=GB&_stpos=BS81QU&orig_cvip=true'''
                elif input_.loc[a, 'Market'] == 'AU':
                    url = f'''https://www.ebay.com.au/itm/{input_.loc[a, 'Item Number']}?_ul=AU&_stpos=2019&orig_cvip=true'''

                resp = requests.get(url,
                                    proxies=get_proxy(),
                                    timeout=(10, 10)).text

                # = = = = = = = = = = = = = = =

                import json

                json_ = json.loads(resp.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script>')[0].strip() + ']}]}')

                # = = = = = = = = = = = = = = =

                market = json_['o']['g']['marketplaceId'].strip()
                country = json_['o']['g']['country'].strip()
                language = json_['o']['g']['language'].strip()
                locale = json_['o']['g']['locale'].strip()
                item_number = json_['o']['g']['itemId'].strip()

                # = = = = = = = = = = = = = = =

                if input_.loc[a, 'Market'] == 'US':
                    if item_number != input_.loc[a, 'Item Number'] or market != 'EBAY-US' or country != 'US' or language != 'en' or locale != 'en-US':
                        continue
                if input_.loc[a, 'Market'] == 'DE':
                    if item_number != input_.loc[a, 'Item Number'] or market != 'EBAY-DE' or country != 'DE' or language != 'de' or locale != 'de-DE':
                        continue
                if input_.loc[a, 'Market'] == 'UK':
                    if item_number != input_.loc[a, 'Item Number'] or market != 'EBAY-GB' or country != 'GB' or language != 'en' or locale != 'en-GB':
                        continue
                if input_.loc[a, 'Market'] == 'AU':
                    if item_number != input_.loc[a, 'Item Number'] or market != 'EBAY-AU' or country != 'AU' or language != 'en' or locale != 'en-AU':
                        continue

                # = = = = = = = = = = = = = = =

                list_compatibility = json_['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['pages'] if 'COMPATIBILITY_TABLE' in json_['o']['w'][0][2]['model']['modules'] else []

                dict_compatibility = {'market': input_.loc[a, 'Market'],
                                      'page': int(list_compatibility[-1]['text'].strip()),
                                      'data': {'scopedContext': {'catalogDetails': {'categoryId': list_compatibility[-1]['action']['params']['categoryId'],
                                                                                    'itemId': list_compatibility[-1]['action']['params']['itemId'],
                                                                                    'marketplaceId': list_compatibility[-1]['action']['params']['marketplaceId'],
                                                                                    'sellerName': list_compatibility[-1]['action']['params']['sellerName']}}}} if list_compatibility else ''

                # = = = = = = = = = = = = = = =

                dict_specific = {}
                dict_item = json_['o']['w'][0][2]['model']['modules']['ABOUT_THIS_ITEM']['sections']['features']['dataItems']
                for i, (key, value) in zip(range(len(dict_item.items())), dict_item.items()):
                    dict_specific[str(i)] = {value['labels'][0]['textSpans'][0]['text'].strip(): value['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip() if key.strip().lower() == 'condition' else value['values'][0]['textSpans'][0]['text'].strip()}

                # = = = = = = = = = = = = = = =

                description_url = json_['o']['w'][0][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['action']['URL'].strip()

                # = = = = = = = = = = = = = = =

                title = json_['o']['w'][0][2]['model']['modules']['TITLE']['mainTitle']['textSpans'][0]['text'].strip()

                # = = = = = = = = = = = = = = =

                subtitle = json_['o']['w'][0][2]['model']['modules']['TITLE']['subTitle']['textSpans'][0]['text'].strip() if 'subTitle' in json_['o']['w'][0][2]['model']['modules']['TITLE'] else ''

                # = = = = = = = = = = = = = = =

                price = json_['o']['w'][0][2]['model']['modules']['BUY_BOX']['binModel']['price']['textSpans'][0]['text'].strip()

                # = = = = = = = = = = = = = = =

                available, sold = '', ''
                list_quantity = json_['o']['w'][0][2]['model']['modules']['QUANTITY']['availabilitySignal']['textSpans'] if 'QUANTITY' in json_['o']['w'][0][2]['model']['modules'] else []
                for quantity in list_quantity:
                    if input_.loc[a, 'Market'] in ['US', 'UK', 'AU']:
                        if 'available' in quantity['text']:
                            available = quantity['text'].strip()
                        elif 'sold' in quantity['text']:
                            sold = quantity['text'].strip()
                    elif input_.loc[a, 'Market'] == 'DE':
                        if 'verfügbar' in quantity['text']:
                            available = quantity['text'].strip()
                        elif 'verkauft' in quantity['text']:
                            sold = quantity['text'].strip()

                # = = = = = = = = = = = = = = =

                dict_src = {}
                list_media = json_['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList']
                for i, media in zip(range(len(list_media)), list_media):
                    dict_src[str(i)] = media['image']['originalImg']['URL'].strip()

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Item Number': input_.loc[a, 'Item Number'],
                                         'Title': title,
                                         'Subtitle': subtitle,
                                         'Price': price,
                                         'Available': available,
                                         'Sold': sold,
                                         'Vehicle W/o Engine': '',
                                         'Vehicle W/ Engine': '',
                                         'Picture': '',
                                         'Url': url,
                                         'Description Url': description_url,
                                         'Json_Compatibility': json.dumps(dict_compatibility),
                                         'Json_Src': json.dumps(dict_src),
                                         'Json_Specific': json.dumps(dict_specific)}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'

                # = = = = = = = = = = = = = = =

                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Market': input_.loc[a, 'Market'],
                                     'Page': input_.loc[a, 'Page'],
                                     'Page Url': input_.loc[a, 'Page Url'],
                                     'No': input_.loc[a, 'No'],
                                     'Item Number': input_.loc[a, 'Item Number']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b+1:3d}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Item Number']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if len(output_correct) != 0:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./3.item-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if len(output_error) != 0:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True, True],
                                            ignore_index=True)
    output_error.to_excel(f'./item_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：366

[状态：ok，尝试次数：  1] - 9.192469913773
[剩余数量：354] - [当前时间：20:44:58]

[状态：ok，尝试次数：  1] - 6.192470816410
[剩余数量：353] - [当前时间：20:44:58]

[状态：ok，尝试次数：  1] - 11.196084651217
[剩余数量：352] - [当前时间：20:44:59]

[状态：ok，尝试次数：  1] - 5.192470815384
[剩余数量：351] - [当前时间：20:44:59]

[状态：ok，尝试次数：  1] - 2.195660971887
[剩余数量：350] - [当前时间：20:44:59]

[状态：ok，尝试次数：  1] - 14.193432073431
[剩余数量：349] - [当前时间：20:44:59]

[状态：ok，尝试次数：  1] - 13.193432073395
[剩余数量：348] - [当前时间：20:45:00]

[状态：ok，尝试次数：  1] - 8.192399272603
[剩余数量：347] - [当前时间：20:45:00]

[状态：ok，尝试次数：  1] - 15.193432073468
[剩余数量：346] - [当前时间：20:45:00]

[状态：ok，尝试次数：  1] - 16.192461026915
[剩余数量：345] - [当前时间：20:45:00]

[状态：ok，尝试次数：  1] - 20.192461038502
[剩余数量：344] - [当前时间：20:45:02]

[状态：ok，尝试次数：  1] - 3.192557565418
[剩余数量：343] - [当前时间：20:45:02]

[状态：ok，尝试次数：  1] - 19.192461036905
[剩余数量：342] - [当前时间：20:45:02]

[状态：ok，尝试次数：  1] - 17.192461027622
[剩余数量：341] - [当前时间：20:45:02]

[状态：ok，尝试次数：  1] - 21.192461038654
[剩余数量：340] - [当前时间：20:45:02]

[状态：ok，尝试次数：  3] - 12.